### データをimport

Anchorageは発着地から削除されている

In [46]:
import pandas as pd
import networkx as nx

# Sample data creation
data = pd.read_csv("data/input/input.csv")

df = pd.DataFrame(data)

### 港名と国名の辞書を作成

In [47]:
port2country = dict(zip(df['Source'], df['Source_Country']))
port2country

{"'s-Gravendeel": 'Netherlands',
 'Aalborg': 'Denmark',
 'Aalst': 'Belgium',
 'Aarhus': 'Denmark',
 'Aasiaat (Egedesminde)': 'Greenland',
 'Abenra': 'Denmark',
 "Aber Wrac'h": 'France',
 'Aberdeen': 'U.K.',
 'Abidjan': 'Ivory Coast',
 'Abu Al Fulus': 'Iraq',
 'Abu Kir': 'Egypt',
 'Acajutla': 'El Salvador',
 'Acapulco': 'Mexico',
 'AcuPort/Sao Joao da Barra': 'Brazil',
 'Ad Dammam': 'Saudi Arabia',
 'Adamas Milos': 'Greece',
 'Adelaide': 'Australia',
 'Aden': 'Yemen',
 'Agadir': 'Morocco',
 'Agio Theodoroi': 'Greece',
 'Agotnes': 'Norway',
 'Ahus': 'Sweden',
 'Aiambak': 'Papua New Guinea',
 'Ain Sukhna': 'Egypt',
 'Ajman': 'United Arab Emirates',
 'Akita': 'Japan',
 'Akureyri': 'Iceland',
 'Al Adabiyah': 'Egypt',
 'Al Hamriyah': 'United Arab Emirates',
 'Al Jubail Commercial Port': 'Saudi Arabia',
 'Al Khums': 'Libya',
 'Alameda': 'U.S.A.',
 'Alesund': 'Norway',
 'Alexandroupolis': 'Greece',
 'Algeciras': 'Spain',
 'Alger (Algiers)': 'Algeria',
 'Alicante': 'Spain',
 'Alkmaar': 'Netherl

### データフレームを受け取り、グラフ（ネットワーク）と中心性指標を返す関数

In [48]:
import pandas as pd
import networkx as nx

# Define the function to calculate centrality metrics
def calculate_centrality(df):
    # Creating a directed graph
    G = nx.DiGraph()
    
    # Add edges to the graph
    for _, row in df.iterrows():
        G.add_edge(row['Source'], row['Target'], weight=row['Total_TEU_Capacity'])
    
    # Calculating centrality metrics
    eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=100)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    
    # Creating a dataframe to display results
    centrality_df = pd.DataFrame({
        'Eigenvector Centrality': eigenvector_centrality,
        'Betweenness Centrality': betweenness_centrality,
        })
    centrality_df['Source_Country'] = centrality_df.index.map(port2country)
    return G, centrality_df

### 指定した国がSource_Country | Target_Countryに含まれている場合、その行を消す関数

In [49]:
def filter_data_by_country(df, country_name):
    """
    指定された国名を含むSource_CountryまたはTarget_Countryの行を除外する関数
    
    Parameters:
    - df: フィルタリングするデータフレーム
    - country_name: 除外する国名
    
    Returns:
    - フィルタリングされたデータフレーム
    """
    # 「Source_Country」または「Target_Country」に指定された国名が含まれている行を除外
    df_filtered = df[~((df['Source_Country'].str[:6] == country_name) | (df['Target_Country'].str[:6] == country_name))]
    
    # 必要なカラムだけを抽出
    columns = ['Source', 'Target', 'Total_TEU_Capacity', 
               'Source_Country', 'Target_Country']
    df_filtered = df_filtered[columns]
    
    return df_filtered

### input.csvに対して、中心性指標を計算

In [50]:
# 1分くらいかかります
df_G, centrality_df = calculate_centrality(df)
print(centrality_df.head(10))
centrality_df.to_csv("data/output/centralities/centrality.csv", index=True)
centrality_df.info()

               Eigenvector Centrality  Betweenness Centrality Source_Country
's-Gravendeel                0.000005                0.000250    Netherlands
Amsterdam                    0.000196                0.019176    Netherlands
Antwerpen                    0.032727               24.638153        Belgium
Bergambacht                  0.000002                0.000024    Netherlands
Bremerhaven                  0.016656                0.006283        Germany
Gohyeon/Geoje                0.000038                0.000000    South Korea
Moerdijk                     0.000418                0.082980    Netherlands
Rotterdam                    0.085456                0.113012    Netherlands
Aalborg                      0.000022                0.011188        Denmark
Aarhus                       0.000304                0.000633        Denmark
<class 'pandas.core.frame.DataFrame'>
Index: 1926 entries, 's-Gravendeel to Yoshiumi
Data columns (total 3 columns):
 #   Column                  Non-Nul

### input_without_Taiwan.csvを作成

In [51]:
country_name = "Taiwan"

df_without_Taiwan = filter_data_by_country(df, country_name)
# Proceed with the centrality calculation
print(df_without_Taiwan.head(10))
df_without_Taiwan.to_csv("data/input/input_without_Taiwan.csv", index=False)

          Source         Target  Total_TEU_Capacity Source_Country  \
0  's-Gravendeel  's-Gravendeel                 905    Netherlands   
1  's-Gravendeel      Amsterdam                1800    Netherlands   
2  's-Gravendeel      Antwerpen               25800    Netherlands   
3  's-Gravendeel    Bergambacht                1800    Netherlands   
4  's-Gravendeel    Bremerhaven                 905    Netherlands   
5  's-Gravendeel  Gohyeon/Geoje               15000    Netherlands   
6  's-Gravendeel       Moerdijk                 508    Netherlands   
7  's-Gravendeel      Rotterdam               23060    Netherlands   
8        Aalborg        Aalborg                2150        Denmark   
9        Aalborg         Aarhus               25720        Denmark   

  Target_Country  
0    Netherlands  
1    Netherlands  
2        Belgium  
3    Netherlands  
4        Germany  
5    South Korea  
6    Netherlands  
7    Netherlands  
8        Denmark  
9        Denmark  


### input_without_Taiwan.csvに対して、中心性指標を計算

In [52]:
# 1分くらいかかります
df_G_no_Taiwan, centrality_no_taiwan_df = calculate_centrality(df_without_Taiwan)
print(centrality_no_taiwan_df.head(10))
centrality_no_taiwan_df.to_csv("data/output/centralities/centrality_without_Taiwan.csv", index=True)
centrality_no_taiwan_df.info()

               Eigenvector Centrality  Betweenness Centrality Source_Country
's-Gravendeel                0.000005                0.000252    Netherlands
Amsterdam                    0.000212                0.019248    Netherlands
Antwerpen                    0.035744               24.600420        Belgium
Bergambacht                  0.000002                0.000024    Netherlands
Bremerhaven                  0.018220                0.006328        Germany
Gohyeon/Geoje                0.000038                0.000000    South Korea
Moerdijk                     0.000454                0.083588    Netherlands
Rotterdam                    0.090187                0.113357    Netherlands
Aalborg                      0.000024                0.011261        Denmark
Aarhus                       0.000339                0.000637        Denmark
<class 'pandas.core.frame.DataFrame'>
Index: 1918 entries, 's-Gravendeel to Yoshiumi
Data columns (total 3 columns):
 #   Column                  Non-Nul

### 台湾を消す前後での中心性指標を比較

In [53]:
centrality_df

,Eigenvector Centrality,Betweenness Centrality,Source_Country
's-Gravendeel,4.645296e-06,2.498623e-04,Netherlands
Amsterdam,1.964939e-04,1.917629e-02,Netherlands
Antwerpen,3.272691e-02,2.463815e+01,Belgium
Bergambacht,1.990164e-06,2.350028e-05,Netherlands
Bremerhaven,1.665613e-02,6.283109e-03,Germany
...,...,...,...
Valldal,1.206991e-14,0.000000e+00,Norway
Villa Oliva,1.193732e-13,2.689203e-04,Paraguay
Wemeldinge,1.308940e-32,2.741567e-07,Netherlands
Wuxue Port,3.596766e-10,0.000000e+00,China


In [54]:
centrality_no_taiwan_df

,Eigenvector Centrality,Betweenness Centrality,Source_Country
's-Gravendeel,5.051265e-06,2.519470e-04,Netherlands
Amsterdam,2.118173e-04,1.924806e-02,Netherlands
Antwerpen,3.574375e-02,2.460042e+01,Belgium
Bergambacht,2.171389e-06,2.369614e-05,Netherlands
Bremerhaven,1.821998e-02,6.327721e-03,Germany
...,...,...,...
Valldal,1.391738e-14,0.000000e+00,Norway
Villa Oliva,1.357972e-13,2.684475e-04,Paraguay
Wemeldinge,1.933397e-32,2.764464e-07,Netherlands
Wuxue Port,3.634117e-10,0.000000e+00,China


#### 媒介中心性の差に着目

In [55]:
import pandas as pd

df_1 = centrality_df
df_2 = centrality_no_taiwan_df

# 空のリストを作成して後でデータフレームに追加
change_data = []

# df_1 のインデックスをループして、df_2 のインデックスが一致する場合に計算
for index in df_1.index:
    if index in df_2.index:  # df_2 に存在するインデックスにのみ処理
        # df_1 と df_2 の Betweenness Centrality の差を計算
        df_betweenness_value = df_1.loc[index, 'Betweenness Centrality']
        no_taiwan_betweenness_value = df_2.loc[index, 'Betweenness Centrality']
        
        # 国名を取得
        df_country = df_1.loc[index, 'Source_Country']
        
        # Betweenness Centrality の差を計算
        betweenness_diff = (no_taiwan_betweenness_value - df_betweenness_value)
        
        # データを追加
        change_data.append({
            "Index": index,
            "Country": df_country,
            "Betweenness Centrality Difference": betweenness_diff
        })

# 変化があったデータをデータフレームに変換
change_betweenness_df = pd.DataFrame(change_data)
change_betweenness_df_sorted = change_betweenness_df.sort_values(by='Betweenness Centrality Difference', ascending=False)

# 結果表示
print(change_betweenness_df_sorted)
change_betweenness_df_sorted.to_csv("data/output/changes/change_betweenness.csv", index=False)

               Index    Country  Betweenness Centrality Difference
872          Bangkok   Thailand                          38.094256
967          Qingdao      China                          25.462020
1050  Fangcheng Port      China                          20.727368
828         Bahudopi  Indonesia                          14.468700
200    Zhoushan Port      China                          11.319280
...              ...        ...                                ...
952      Dafeng Port      China                          -0.387642
268         Nakhodka     Russia                          -0.568847
310        Rongcheng      China                          -0.649384
973   Zhangzhou Port      China                          -0.661388
948      Shidao Port      China                         -15.515063

[1918 rows x 3 columns]


#### 固有ベクトル中心性の差に着目

In [56]:
import pandas as pd

df_1 = centrality_df
df_2 = centrality_no_taiwan_df

# 空のリストを作成して後でデータフレームに追加
change_data = []

# df_1 のインデックスをループして、df_2 のインデックスが一致する場合に計算
for index in df_1.index:
    if index in df_2.index:  # df_2 に存在するインデックスにのみ処理
        # df_1 と df_2 の Betweenness Centrality の差を計算
        df_eigenvector_value = df_1.loc[index, 'Eigenvector Centrality']
        no_taiwan_eigenvector_value = df_2.loc[index, 'Eigenvector Centrality']
        
        # 国名を取得
        df_country = df_1.loc[index, 'Source_Country']
        
        # Betweenness Centrality の差を計算
        eigenvector_diff = (no_taiwan_eigenvector_value - df_eigenvector_value)
        
        # データを追加
        change_data.append({
            "Index": index,
            "Country": df_country,
            "Eigenvector Centrality Difference": eigenvector_diff
        })

# 変化があったデータをデータフレームに変換
change_eigenvector_df = pd.DataFrame(change_data)
change_eigenvector_df_sorted = change_eigenvector_df.sort_values(by='Eigenvector Centrality Difference', ascending=False)
print(change_eigenvector_df_sorted.head(10))
# 結果表示
print(change_eigenvector_df_sorted.columns)
change_eigenvector_df_sorted.to_csv("data/output/changes/change_eigenvector.csv", index=False)

                              Index               Country  \
140    Port Klang (Pelabuhan Klang)              Malaysia   
103                         Colombo             Sri Lanka   
148                       Singapore             Singapore   
117  Jawaharlal Nehru (Nhava Sheva)                 India   
161                       Jebel Ali  United Arab Emirates   
7                         Rotterdam           Netherlands   
152                 Tanjung Pelepas              Malaysia   
231                    Laem Chabang              Thailand   
151                      Tanger Med               Morocco   
2                         Antwerpen               Belgium   

     Eigenvector Centrality Difference  
140                           0.038077  
103                           0.006822  
148                           0.005807  
117                           0.005303  
161                           0.004920  
7                             0.004731  
152                           0.004465  


### 中心性指標の差（台湾を消す前後）を国ごとに集計し、降順に並べる

#### 媒介中心性

In [57]:
# 'Country (df)' でグループ化し、'Betweenness Centrality Difference' の国ごとの和を求める
change_betweenness_df_groupby_country = change_betweenness_df.groupby('Country').agg({
    'Betweenness Centrality Difference': 'sum'  # 合計を計算
})

# 'Betweenness Centrality Difference' を基に降順で並べ替え
change_betweenness_df_groupby_country_sorted = change_betweenness_df_groupby_country.sort_values(by='Betweenness Centrality Difference', ascending=False)

# 結果表示
print(change_betweenness_df_groupby_country_sorted)
# 結果をCSVに保存
change_betweenness_df_groupby_country_sorted.to_csv("data/output/change_betweenness_df_groupby_country.csv", index=True)

                          Betweenness Centrality Difference
Country                                                    
China                                             75.731554
Thailand                                          41.972755
Indonesia                                         14.469077
Germany                                            0.238148
Hong Kong, S.A.R., China                           0.027178
...                                                     ...
Iran                                              -0.000317
Japan                                             -0.009136
South Korea                                       -0.028252
Belgium                                           -0.037641
Russia                                            -0.568590

[175 rows x 1 columns]


#### 固有ベクトル中心性

In [58]:
change_eigenvector_df_groupby_country = change_eigenvector_df_sorted.groupby('Country').agg({
    'Eigenvector Centrality Difference': 'sum'  # 合計を計算
})

change_eigenvector_df_groupby_country_sorted = change_eigenvector_df_groupby_country.sort_values(by='Eigenvector Centrality Difference', ascending=False)

# 結果表示
print(change_eigenvector_df_groupby_country_sorted)
# 結果をCSVに保存
change_eigenvector_df_groupby_country_sorted.to_csv("data/output/change_eigenvector_df_groupby_country.csv", index=True)

                          Eigenvector Centrality Difference
Country                                                    
Malaysia                                           0.046024
India                                              0.013421
Sri Lanka                                          0.006837
United Arab Emirates                               0.006826
Singapore                                          0.005812
...                                                     ...
U.S.A.                                            -0.008271
Panama                                            -0.008872
Japan                                             -0.009474
Hong Kong, S.A.R., China                          -0.029506
China                                             -0.030982

[175 rows x 1 columns]
